## Server MAS
Adapted by [Jorge Cruz](https://jcrvz.co) for:
TC2008B. Sistemas Multiagentes y Gráficas Computacionales. Tecnológico de Monterrey.

> Revised version, Nov. 2021

> Original implementation: Python server to interact with Unity, Sergio Ruiz, Jul. 2021

In [ ]:
# Load the local driver (you need to activate your Drive in Colab)
%cd "/content/drive/MyDrive/TC2008B/G4/"

# Size of the board:
width, height = 50, 50

In [2]:
# Install pyngrok to propagate the http server
%pip install pyngrok --quiet

# Load the required packages
from pyngrok import ngrok
from http.server import BaseHTTPRequestHandler, HTTPServer
import logging
import json
import os
import numpy as np

# Boid is a dummy implementation that can be replaced with a more sophisticated 
# MAS module, for example, MESA
from boid import Boid

# Start ngrok
ngrok.install_ngrok()

# Terminate open tunnels if exist
ngrok.kill()

# Open an HTTPs tunnel on port 8585 for http://localhost:8585
port = os.environ.get("PORT", 8585)
server_address = ("", port)

public_url = ngrok.connect(port="8585", proto="http", options={"bind_tls": True})
print("\n" + "#" * 94)
print(f"## Tracking URL: {public_url} ##")
print("#" * 94, end="\n\n")


##############################################################################################
## Tracking URL: NgrokTunnel: "http://7fb3-35-245-218-31.ngrok.io" -> "http://localhost:80" ##
##############################################################################################



In [3]:
ngrok.kill()

# Set the number of agents here:
num_agents = 30

flock = [Boid(*np.random.rand(2) * num_agents, width, height) 
  for _ in range(num_agents)]

flock_class = ['Green', 'Red'] * (30 // 2)
np.random.shuffle(flock_class)

# The way how agents are updated (per step/iteration)
def updateFeatures():
    global flock
    features = []

    # For each agent...
    for boid, colour in zip(flock, flock_class):
        # Update its state
        boid.apply_behaviour(flock)
        boid.update()

        # Read its features
        position = list(boid.edges())        
        features.append([{"x": position[0], "y": position[2], "z": position[1]},
                         'Ball', colour])

    return features

# Post the information in `features` for each iteration
def featuresToJSON(info_list):
    featureDICT = []
    for info in info_list:
        feature = {
            "position" : info[0], # position
            "kind" : info[1], # kind
            "colour" : info[2] # colour
        }
        featureDICT.append(feature)
    return json.dumps(featureDICT)

# This is the server. It controls the simulation.
# Server run (do not change it)
class Server(BaseHTTPRequestHandler):
    
    def _set_response(self):
        self.send_response(200)
        self.send_header('Content-type', 'text/html')
        self.end_headers()
        
    def do_GET(self):
        logging.info("GET request,\nPath: %s\nHeaders:\n%s\n", 
                     str(self.path), str(self.headers))
        self._set_response()
        self.wfile.write("GET request for {}".format(self.path).encode('utf-8'))

    def do_POST(self):
        content_length = int(self.headers['Content-Length'])

        #post_data = self.rfile.read(content_length)
        post_data = json.loads(self.rfile.read(content_length))
        
        # If you have issues with the encoder, toggle the following lines: 
        #logging.info("POST request,\nPath: %s\nHeaders:\n%s\n\nBody:\n%s\n",
                     #str(self.path), str(self.headers), post_data.decode('utf-8'))
        logging.info("POST request,\nPath: %s\nHeaders:\n%s\n\nBody:\n%s\n",
                     str(self.path), str(self.headers), json.dumps(post_data))

        # Here, magick happens 
        # --------------------       
        features = updateFeatures()
        #print(features)

        self._set_response()
        resp = "{\"data\":" + featuresToJSON(features) + "}"
        #print(resp)

        self.wfile.write(resp.encode('utf-8'))

# Server run (do not change it)
def run(server_class=HTTPServer, handler_class=Server, port=8585):
    logging.basicConfig(level=logging.INFO)
    server_address = ('', port)
    httpd = server_class(server_address, handler_class)

    public_url = ngrok.connect(port).public_url
    logging.info("ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}\"".format(
        public_url, port))

    logging.info("Starting httpd...\n") # HTTPD is HTTP Daemon!
    try:
        httpd.serve_forever()
    except KeyboardInterrupt:   # CTRL + C stops the server
        pass

    httpd.server_close()
    logging.info("Stopping httpd...\n")

In [4]:
run(HTTPServer, Server)

INFO:pyngrok.process.ngrok:t=2021-11-14T21:30:14+0000 lvl=info msg="join connections" obj=join id=c37407db3e9d l=127.0.0.1:8585 r=187.161.115.63:10185
INFO:root:POST request,
Path: /
Headers:
Host: 5519-35-245-218-31.ngrok.io
User-Agent: UnityPlayer/2021.2.1f1 (UnityWebRequest/1.0, libcurl/7.75.0-DEV)
Content-Length: 97
Accept: */*
Accept-Encoding: deflate, gzip
Content-Type: application/json
X-Forwarded-For: 187.161.115.63
X-Forwarded-Proto: http
X-Unity-Version: 2021.2.1f1



Body:
{"kind": "Ball", "colour": "Red", "position": {"x": 3.440000057220459, "y": 0.0, "z": -15.706999778747559}}

127.0.0.1 - - [14/Nov/2021 21:30:14] "POST / HTTP/1.1" 200 -
INFO:pyngrok.process.ngrok:t=2021-11-14T21:30:19+0000 lvl=info msg="join connections" obj=join id=06b41f8fdeb9 l=127.0.0.1:8585 r=187.161.115.63:10185
INFO:root:POST request,
Path: /
Headers:
Host: 5519-35-245-218-31.ngrok.io
User-Agent: UnityPlayer/2021.2.1f1 (UnityWebRequest/1.0, libcurl/7.75.0-DEV)
Content-Length: 97
Accept: */*
Accept-